In [4]:
from Tool_Functions.cleaning_data import *
from Tool_Functions.comportment_reabo import * 
from Tool_Functions.join_data import * 

In [5]:
path_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"
path_results_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Results/"

Nouveaux indicateurs : 
- Mois du réabonnement 
- Delai de réabonnement moyen pour cette promo comparé au délai de réabonnement moyen pour toutes les promos
- Trouver aussi un moyen de quantifier le poids de l’abonnement par rapport aux habitudes de fidélité d’un client : s’il s’abonne alors qu’il s’abonne jamais, la promo aura un poids plus important

In [110]:
def enlever_abos(data_path, data_path_results):
    df = file_to_dataframe(data_path + "df_Donnees_Reabos_odd.csv")
    df['NB_APPARITIONS'] = df.groupby('ID_ABONNE')['ID_ABONNE'].transform('count')
    df_new = df[(df['NB_APPARITIONS'] < 34) & (df['NB_APPARITIONS'] > 2)]
    save_to_csv_file(df_new, data_path_results + 'df_Donnees_Reabos_odd_new.csv')
     
    return True

In [165]:
def ajouter_differences(df):
    
    promo_types = [col for col in df.columns if 'MEAN_TIME' in col and col != 'MOY_DELAI']
    
    for promo in promo_types:
        new_col_name = promo + '_DIFF'
        df[new_col_name] = df[promo] - df['MOY_DELAI']

    return df

In [174]:
def create_new_data_set_n_month(data_path, data_path_results):
    """
    Creates a new dataset 'new_data_set' from 'df_Données_Reabos_odd' dataframe where the columns are the 
    TYPE_PROMON
    """

    # Load 'df_Données_Reabos_odd' dataframe
    df = file_to_dataframe(data_path + "df_Donnees_Reabos_odd_new.csv")
    df['DATE_ACTE_REEL'] = pd.to_datetime(df['DATE_ACTE_REEL'])
    df['MONTH'] = df['DATE_ACTE_REEL'].dt.month

    # Perform statistical analysis on 'df_Données_Reabos_odd' dataframe
    df2 = count_abo_conditions(df, ['ID_ABONNE', 'MONTH'], 'DATE_ACTE_REEL')

    # Create a pivot table from 'df' dataframe we exchange the lines and columns
    new_data_set = df2.pivot_table(index='ID_ABONNE', columns='MONTH', values='NB_DATE_ACTE_REEL')

    # Reset index of the pivot table
    new_data_set = new_data_set.reset_index()

    # Replace NaN values with 0
    new_data_set.fillna(0, inplace=True)

    # Convert floating-point values to integers
    new_data_set = new_data_set.astype(int)

    # Save 'new_data_set' dataframe to a CSV file
    save_to_csv_file(new_data_set, data_path_results + "df_n_months_ODD.csv")

    return True

In [175]:
create_new_data_set_n_month(path_antoine, path_antoine)

True

In [178]:
def create_new_data_set_n_month_pourc(data_path, data_path_results):
    """
    Creates a new dataset 'new_data_set' from 'df_Données_Reabos_odd' dataframe where the columns are the 
    TYPE_PROMON
    """

    # Load 'df_Données_Reabos_odd' dataframe
    df = file_to_dataframe(data_path + "df_Donnees_Reabos_odd_new.csv")
    df['DATE_ACTE_REEL'] = pd.to_datetime(df['DATE_ACTE_REEL'])
    df['MONTH'] = df['DATE_ACTE_REEL'].dt.month

    # Perform statistical analysis on 'df_Données_Reabos_odd' dataframe
    df2 = count_abo_conditions(df, ['ID_ABONNE', 'MONTH'], 'DATE_ACTE_REEL')

    # Create a pivot table from 'df' dataframe we exchange the lines and columns
    new_data_set = df2.pivot_table(index='ID_ABONNE', columns='MONTH', values='NB_DATE_ACTE_REEL')

    new_data_set_pourc = new_data_set.apply(lambda x: (x / x.sum()) * 100, axis=1)

    # Reset index of the pivot table
    new_data_set_pourc = new_data_set_pourc.reset_index()

    # Replace NaN values with 0
    new_data_set_pourc.fillna(0, inplace=True)

    # Convert floating-point values to integers
    new_data_set_pourc = new_data_set_pourc.astype(int)

    # Save 'new_data_set' dataframe to a CSV file
    save_to_csv_file(new_data_set_pourc, data_path + "df_n_months_ODD_%.csv")

    return True

In [179]:
create_new_data_set_n_month_pourc(path_antoine, path_antoine)

True

In [180]:
def create_new_data_set_n_fidelite(data_path, data_path_results):
    """
    Creates a new dataset 'new_data_set' from 'df_Données_Reabos_odd' dataframe where the columns are the 
    TYPE_PROMON
    """

    # Load 'df_Données_Reabos_odd' dataframe
    df = file_to_dataframe(data_path + "df_Donnees_Reabos_odd_new.csv")
    df['DATE_ACTE_REEL'] = pd.to_datetime(df['DATE_ACTE_REEL'])
    df['MONTH'] = df['DATE_ACTE_REEL'].dt.month

    # Perform statistical analysis on 'df_Données_Reabos_odd' dataframe
    df2 = count_abo_conditions(df, ['ID_ABONNE', 'STATUT_FIN_M_MOINS_1'], 'DATE_ACTE_REEL')

    # Create a pivot table from 'df' dataframe we exchange the lines and columns
    new_data_set = df2.pivot_table(index='ID_ABONNE', columns='STATUT_FIN_M_MOINS_1', values='NB_DATE_ACTE_REEL')

    # Reset index of the pivot table
    new_data_set = new_data_set.reset_index()

    # Replace NaN values with 0
    new_data_set.fillna(0, inplace=True)

    # Convert floating-point values to integers
    new_data_set = new_data_set.astype(int)

    # Save 'new_data_set' dataframe to a CSV file
    save_to_csv_file(new_data_set, data_path_results + "df_n_fidélité_ODD.csv")

    return True

In [181]:
create_new_data_set_n_fidelite(path_antoine, path_antoine)

True

In [ ]:
# Combien de fois ils se sont abonnés dans l'année

In [ ]:
def create_new_data_set_n_abo(data_path, data_path_results):
    """
    Creates a new dataset 'new_data_set' from 'df_Données_Reabos_odd' dataframe where the columns are the 
    TYPE_PROMON
    """

    # Load 'df_Données_Reabos_odd' dataframe
    df = file_to_dataframe(data_path + "df_Donnees_Reabos_odd_new.csv")
    df['DATE_ACTE_REEL'] = pd.to_datetime(df['DATE_ACTE_REEL'])
    df['MONTH'] = df['DATE_ACTE_REEL'].dt.month

    # Perform statistical analysis on 'df_Données_Reabos_odd' dataframe
    df2 = count_abo_conditions(df, ['ID_ABONNE', 'STATUT_FIN_M_MOINS_1'], 'DATE_ACTE_REEL')

    # Create a pivot table from 'df' dataframe we exchange the lines and columns
    new_data_set = df2.pivot_table(index='ID_ABONNE', columns='STATUT_FIN_M_MOINS_1', values='NB_DATE_ACTE_REEL')

    # Reset index of the pivot table
    new_data_set = new_data_set.reset_index()

    # Replace NaN values with 0
    new_data_set.fillna(0, inplace=True)

    # Convert floating-point values to integers
    new_data_set = new_data_set.astype(int)

    # Save 'new_data_set' dataframe to a CSV file
    save_to_csv_file(new_data_set, data_path_results + "df_n_fidélité_ODD.csv")

    return True

In [6]:
df = file_to_dataframe(path_antoine + "df_Donnees_Reabos_odd_new.csv")